In [ ]:
cd /work/deeptrust/src/continuous-x/
chmod 777 *.sh


Edit clouds.yaml and get leases for baremtal compute

In [ ]:
sed -i 's/'"<REDACTED_RESERVATION_LEASE>"'/'"<LEASE>"'/g' /work/deeptrust/src/continuous-x/tf/tacc/main.tf
sed -i 's/'"<REDACTED_RESERVATION_LEASE>"'/'"<LEASE>"'/g' /work/deeptrust/src/continuous-x/tf/uc/main.tf


In [ ]:
./download-libraries.sh

In [ ]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
PYTHONUSERBASE=/work/.local pip install --user -r /work/deeptrust/src/continuous-x/ansible/k8s/kubespray/requirements.txt


Provision resources using terraform

In [ ]:
./provision-terraform.sh

Configuring kubernetes on the host

In [ ]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml pre_k8s/pre_k8s_configure.yml


In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd k8s/kubespray
ansible-playbook -i ../inventory/mycluster --become --become-user=root ./cluster.yml


In [ ]:
#Change Github Access Token before running
cd ../../ansible
ansible-playbook -i inventory.yml post_k8s/post_k8s_configure.yml

ArgoCD
Attach Block Storage to instance and format it after ssh-ing on the node

sudo parted -s /dev/vdb mklabel gpt
sudo parted -s /dev/vdb mkpart primary ext4 0% 100%
sudo mkfs.ext4 /dev/vdb1

sudo mkdir -p /mnt/block
sudo mount /dev/vdb1 /mnt/block

sudo chown -R cc /mnt/block
sudo chgrp -R cc /mnt/block

Baremetal Setup

1) SSH into hosts
2) Install Docker
3) Install RClone 
    curl https://rclone.org/install.sh | sudo bash
    sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf
    mkdir -p ~/.config/rclone
    nano  ~/.config/rclone/rclone.conf
3) Create rclone.conf
3) Attach object storage
sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
sudo chgrp -R cc /mnt/object
rclone mount chi_tacc:object-persist-netID /mnt/object --read-only --allow-other --daemon

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd ../
ansible-playbook -i ansible/inventory.yml argocd/initial_container_create.yml

Change Repo Token in argocd_add_platform.yml

In [ ]:
cd ../
ansible-playbook -i ansible/inventory.yml argocd/argocd_add_platform.yml

Enable port forwarding for services
ssh -L 8443:127.0.0.1:8443 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D
kubectl port-forward -n kube-system svc/kubernetes-dashboard 8443:443
https://127.0.0.1:8443/

ssh -L 8888:127.0.0.1:8888 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D
kubectl port-forward svc/argocd-server -n argocd 8888:443
https://127.0.0.1:8888/

ssh -L 2746:127.0.0.1:2746 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D
kubectl -n argo port-forward svc/argo-server 2746:2746
https://127.0.0.1:2746/

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible-baremetal/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
ansible-playbook -i ansible/inventory.yml setup/bootstrap.yml